In [0]:
df = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/jan.csv")
df.display()


In [0]:
from pyspark.sql.functions import current_timestamp
df_bronze = df.withColumn("ingesttime", current_timestamp())
df_bronze.write.format("delta").mode("overwrite").save("/mnt/govinda/bronze2")


In [0]:
dbutils.fs.ls("/mnt/govinda")


In [0]:
 # Fix dirty price values
from pyspark.sql.functions import col, trim, length, when

df_silver = df_bronze \
    .withColumn(
        "price",
        when(
            (length(trim(col("price"))) == 0) | (col("price").isin("null", "NA")),
            None
        ).otherwise(col("price")).cast("double")
    )
    
    # Convert timestamp
    .withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss")) \
    
    # Only filter critical
    .filter(col("event_time").isNotNull()) \
    .filter(col("event_type").isNotNull()) \
    
    # Replace nulls instead of filtering
    .withColumn("brand", coalesce(col("brand"), lit("unknown"))) \
    .withColumn("category_code", coalesce(col("category_code"), lit("uncategorized"))) \
    
    # Optional cast
    .withColumn("user_id", col("user_id").cast("long")) \
    .withColumn("product_id", col("product_id").cast("long")) \
    .withColumn("category_id", col("category_id").cast("long")) \
    
    # Add event_date for grouping
    .withColumn("event_date", to_date("event_time"))
display(df_silver)  


In [0]:
#df1 = df_bronze.filter(col("event_time").isNotNull())
#print("After filtering event_time:", df1.count())

#df2 = df1.filter(col("event_type").isNotNull())
#print("After filtering event_type:", df2.count())

#df3 = df2.filter(col("price").isNotNull())
#print("After filtering price:", df3.count())

#df4 = df3.filter(col("category_code").isNotNull())
#print("After filtering category_code:", df4.count())


In [0]:
df5 = df4.filter(col("brand").isNotNull())
print("After filtering brand:", df5.count())

In [0]:
df = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/jan.csv")
df1 = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/feb.csv")
df2 = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/oct.csv")
df3 = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/nov.csv")
df4 = spark.read.option("header", True).csv("/mnt/govinda/Rawavan/ecart/dec.csv")

df_raw = df.unionByName(df1).unionByName(df2).unionByName(df3).unionByName(df4)
df_raw.write.format("delta").mode("overwrite").save("/mnt/govinda/Rama")








In [0]:
print("Total Rows After Merge:", df_raw.count())


In [0]:
from pyspark.sql.functions import current_timestamp

df_bronze = df_raw.withColumn("ingest_time", current_timestamp())

df_bronze.write.format("delta").mode("overwrite").save("/mnt/govinda/bronzelayer/ecommerce_full")


In [0]:
from pyspark.sql.functions import col, trim, length, when, to_timestamp, to_date, lit, coalesce

df_silver = df_bronze \
    .withColumn(
        "price",
        when((trim(col("price")).isin("", "null", "NA")), None)
        .otherwise(col("price")).cast("double")
    ) \
    .withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("product_id", col("product_id").cast("long")) \
    .withColumn("category_id", col("category_id").cast("long")) \
    .withColumn("user_id", col("user_id").cast("long")) \
    .withColumn("brand", coalesce(col("brand"), lit("unknown"))) \
    .withColumn("category_code", coalesce(col("category_code"), lit("uncategorized"))) \
    .withColumn("event_date", to_date("event_time")) \
    .filter(col("event_time").isNotNull()) \
    .filter(col("event_type").isNotNull()) \
    .filter(col("price").isNotNull())


In [0]:
df_bronze.display()

In [0]:
from pyspark.sql.functions import coalesce, lit

df_clean = df_bronze \
    .withColumn("brand", coalesce(col("brand"), lit("unknown"))) \
    .withColumn("category_code", coalesce(col("category_code"), lit("uncategorized")))
df_clean.display()


In [0]:
from pyspark.sql.functions import col

# Keep a temporary cleaned view
#df0= df_clean \
    #.filter(col("event_time").isNotNull()) \
    #.filter(col("event_type").isNotNull()) \
    #.filter(col("price").isNotNull())
#display(df0)
df0.count()


In [0]:
#df0.select("brand", "price", "event_type").show(5)
#df0.groupBy("brand").count().show()


In [0]:
df1 = df0.drop("ingest_time", "category_code")
df2 = df1.filter(col("brand") == "unknown")
df2.write.format("delta").mode("overwrite").save("/mnt/govinda/silverlayer/only_unknown_brands")


In [0]:
display(dbutils.fs.mounts())


In [0]:
dbutils.fs.mount(
  source = "wasbs://sreejachandu001@sreejachandu.blob.core.windows.net/",
  mount_point = "/mnt/govinda",
  extra_configs = {
    "fs.azure.account.key.sreejachandu.blob.core.windows.net": "HFniXOBVYsGPeSOaOpUE0fknhOpQcq7pRLNm6bWnNtiiV3JkJgJX4BHUI1nPU40/9Iw3G7/92X/D+AStWpxqhw=="
  }
)



In [0]:
df2.write.format("delta").mode("overwrite").save("/mnt/govinda/silverlayer/only_unknown_brands")

In [0]:
df0.show(5)

In [0]:
#df1.display()
#df_sliverlayer = df0.drop("category_code")
df_sliverlayer.display()



In [0]:
df_sliverlayer = spark.read.format('delta').load('/mnt/govinda/silverlayer1/allbrand')# or create a DataFrame from existing data, then use df_sliverlayer.write.format('delta').mode('overwrite').save('/mnt/govinda/silverlayer1/allbrand')

df_sliverlayer.schema

In [0]:
from pyspark.sql.functions import *

df4 = df_sliverlayer \
    .withColumn("event_timestamp", to_timestamp(regexp_replace("event_time", " UTC", ""), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("event_date", to_date("event_timestamp")) \
    .withColumn("event_hour", hour("event_timestamp")) \
    .withColumn("event_dayofweek_str", date_format("event_timestamp", "E")) \
    .withColumn("event_dayofweek", 
        when(col("event_dayofweek_str") == "Mon", 1)
       .when(col("event_dayofweek_str") == "Tue", 2)
       .when(col("event_dayofweek_str") == "Wed", 3)
       .when(col("event_dayofweek_str") == "Thu", 4)
       .when(col("event_dayofweek_str") == "Fri", 5)
       .when(col("event_dayofweek_str") == "Sat", 6)
       .when(col("event_dayofweek_str") == "Sun", 7)
       .otherwise(None)) \
    .withColumn("price_numeric", col("price").cast("double")) \
    .withColumn("brand_clean", when(col("brand") == "unknown", None).otherwise(col("brand"))) \
    .withColumn("brand", coalesce(col("brand_clean"), lit("unknown"))) \
    .drop("event_dayofweek_str")  # Optional cleanup
df4.write.format("delta").mode("overwrite").save("/mnt/govinda/silverlayer2/final_clean_silver")






In [0]:
df4.count()

20692840

In [0]:
from pyspark.sql.functions import coalesce, lit

df3 = df4 \
    .withColumn("brand", coalesce(col("brand"), lit("unknown"))) \
    .withColumn("category_code", coalesce(col("category_code"), lit("uncategorized")))
df4.display()

In [0]:
from pyspark.sql.functions import col, sum, desc
# Alias for readability
df = df4

# Golden Layer 1: Top Brands by Volume
df.groupBy("brand").count() \
  .orderBy(desc("count")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/top_brands_by_volume")

# Golden Layer 2: Top Brands by Revenue
df.filter(col("event_type") == "purchase") \
  .groupBy("brand").agg(sum("price_numeric").alias("total_revenue")) \
  .orderBy(desc("total_revenue")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/top_brands_by_revenue")

# Golden Layer 3: Brand Conversion Funnel
df.groupBy("brand", "event_type").count() \
  .groupBy("brand") \
  .pivot("event_type", ["view", "cart", "purchase"]) \
  .sum("count") \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/brand_conversion_funnel")

# Golden Layer 4: Brand Popularity by Day
df.groupBy("event_date", "brand").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/brand_daily_trend")

# Golden Layer 5: Brand Retention (users repeating brands)
df.dropna(subset=["brand"]).dropDuplicates(["user_id", "brand", "event_date"]) \
  .groupBy("brand", "user_id").count().filter("count > 1") \
  .groupBy("brand").count().withColumnRenamed("count", "repeat_users") \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/brand_retention")

# Golden Layer 6: Top Products by Sales
df.filter(col("event_type") == "purchase") \
  .groupBy("product_id").agg(count("*").alias("sales_count")) \
  .orderBy(desc("sales_count")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/top_products_by_sales")

# Golden Layer 7: Most Abandoned Products
views = df.filter(col("event_type") == "cart").groupBy("product_id").count().withColumnRenamed("count", "carted")
purchases = df.filter(col("event_type") == "purchase").groupBy("product_id").count().withColumnRenamed("count", "purchased")
views.join(purchases, "product_id", "left_outer") \
  .withColumn("abandonment", col("carted") - coalesce(col("purchased"), lit(0))) \
  .orderBy(desc("abandonment")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/abandoned_products")

# Golden Layer 8: High Revenue Products
df.filter(col("event_type") == "purchase") \
  .groupBy("product_id").agg(sum("price_numeric").alias("revenue")) \
  .orderBy(desc("revenue")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/high_revenue_products")

# Golden Layer 9: Price Band Distribution
df.withColumn("price_band", when(col("price_numeric") < 10, "<10")
              .when(col("price_numeric").between(10, 50), "10-50")
              .when(col("price_numeric").between(50, 100), "50-100")
              .otherwise(">100")) \
  .groupBy("price_band").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/price_band_distribution")

# Golden Layer 10: Product Lifespan (first to last event)
df.groupBy("product_id").agg(min("event_date").alias("first"), max("event_date").alias("last")) \
  .withColumn("lifespan_days", datediff("last", "first")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/product_lifespan")

# Golden Layer 11: Top Spenders
df.filter(col("event_type") == "purchase") \
  .groupBy("user_id").agg(sum("price_numeric").alias("total_spent")) \
  .orderBy(desc("total_spent")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/top_spenders")

# Golden Layer 12: Avg Session Duration
df.groupBy("user_session").agg(min("event_timestamp").alias("start"), max("event_timestamp").alias("end")) \
  .withColumn("session_duration", unix_timestamp("end") - unix_timestamp("start")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/session_durations")

# Golden Layer 13: Funnel Conversion (Global)
df.groupBy("event_type").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/global_conversion_funnel")

# Golden Layer 14: Repeat Customers
df.filter(col("event_type") == "purchase") \
  .groupBy("user_id").count().filter("count > 1") \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/repeat_customers")

# Golden Layer 15: Sessions with No Purchase
purchased_sessions = df.filter(col("event_type") == "purchase").select("user_session").distinct()
df.select("user_session").distinct().join(purchased_sessions, "user_session", "left_anti") \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/sessions_no_purchase")

# Golden Layer 16: Hourly Activity Heatmap
df.groupBy("event_hour", "event_type").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/hourly_activity")

# Golden Layer 17: Day of Week Activity
df.groupBy("event_dayofweek", "event_type").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/dayofweek_activity")

# Golden Layer 18: Peak Purchase Hours
df.filter(col("event_type") == "purchase") \
  .groupBy("event_hour").agg(sum("price_numeric").alias("revenue")) \
  .orderBy(desc("revenue")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/peak_purchase_hours")

# Golden Layer 19: Monthly Trendline
df.withColumn("month", date_format("event_date", "yyyy-MM")) \
  .groupBy("month", "event_type").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/monthly_trend")

# Golden Layer 20: Weekend vs Weekday
df.withColumn("is_weekend", col("event_dayofweek").isin(6,7)) \
  .groupBy("is_weekend", "event_type").count() \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/weekend_vs_weekday")

# Golden Layer 21: Revenue per Session
df.filter(col("event_type") == "purchase") \
  .groupBy("user_session").agg(sum("price_numeric").alias("revenue")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/revenue_per_session")

# Golden Layer 22: Cart Abandonment Rate
carts = df.filter(col("event_type") == "cart").select("user_session").distinct()
purchases = df.filter(col("event_type") == "purchase").select("user_session").distinct()
carts.join(purchases, "user_session", "left_anti") \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/cart_abandonment_sessions")

# Golden Layer 23: Purchase Frequency by User
df.filter(col("event_type") == "purchase") \
  .groupBy("user_id").agg(count("*").alias("purchase_count")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/user_purchase_frequency")

# Golden Layer 24: Revenue by Category (Fixed to use category_id)
df.filter(col("event_type") == "purchase") \
  .groupBy("category_id") \
  .agg(sum("price_numeric").alias("revenue")) \
  .orderBy(desc("revenue")) \
  .write.mode("overwrite") \
  .format("delta") \
  .save("/mnt/govinda/goldenlayers/category_revenue")
# Golden Layer 25: Time from View to Purchase (basic estimation)
from pyspark.sql.window import Window
w = Window.partitionBy("user_id", "product_id").orderBy("event_timestamp")
df.withColumn("next_event", lead("event_type").over(w)) \
  .withColumn("next_time", lead("event_timestamp").over(w)) \
  .filter((col("event_type") == "view") & (col("next_event") == "purchase")) \
  .withColumn("time_to_purchase_sec", unix_timestamp("next_time") - unix_timestamp("event_timestamp")) \
  .write.mode("overwrite").format("delta").save("/mnt/govinda/goldenlayers/view_to_purchase_time")



In [0]:
df.columns


['event_time',
 'event_type',
 'product_id',
 'category_id',
 'brand',
 'price',
 'user_id',
 'user_session',
 'event_timestamp',
 'event_date',
 'event_hour',
 'event_dayofweek',
 'price_numeric',
 'brand_clean']

In [0]:
%sql CREATE TABLE IF NOT EXISTS monthly_trend
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/monthly_trend'



In [0]:
%sql
show tables

database tableName isTemporary default abandoned_products false default brand_conversion_funnel false default brand_daily_trend false default brand_retention false default cart_abandonment_sessions false default category_revenue false default dayofweek_activity false default global_conversion_funnel false default golden_top_brands_by_volume false default high_revenue_products false default hourly_activity false default monthly_trend false default t1 false default t2 false default v1 false _sqldf true

In [0]:
%sql
CREATE TABLE IF NOT EXISTS golden_top_brands_by_volume
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/top_brands_by_volume';

CREATE TABLE IF NOT EXISTS golden_top_brands_by_revenue
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/top_brands_by_revenue';

CREATE TABLE IF NOT EXISTS golden_brand_conversion_funnel
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/brand_conversion_funnel';

CREATE TABLE IF NOT EXISTS golden_brand_daily_trend
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/brand_daily_trend';

CREATE TABLE IF NOT EXISTS golden_brand_retention
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/brand_retention';

CREATE TABLE IF NOT EXISTS golden_top_products_by_sales
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/top_products_by_sales';

CREATE TABLE IF NOT EXISTS golden_abandoned_products
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/abandoned_products';

CREATE TABLE IF NOT EXISTS golden_high_revenue_products
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/high_revenue_products';

CREATE TABLE IF NOT EXISTS golden_price_band_distribution
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/price_band_distribution';

CREATE TABLE IF NOT EXISTS golden_product_lifespan
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/product_lifespan';

CREATE TABLE IF NOT EXISTS golden_top_spenders
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/top_spenders';

CREATE TABLE IF NOT EXISTS golden_session_durations
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/session_durations';

CREATE TABLE IF NOT EXISTS golden_global_conversion_funnel
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/global_conversion_funnel';

CREATE TABLE IF NOT EXISTS golden_repeat_customers
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/repeat_customers';

CREATE TABLE IF NOT EXISTS golden_sessions_no_purchase
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/sessions_no_purchase';

CREATE TABLE IF NOT EXISTS golden_hourly_activity
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/hourly_activity';

CREATE TABLE IF NOT EXISTS golden_dayofweek_activity
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/dayofweek_activity';

CREATE TABLE IF NOT EXISTS golden_peak_purchase_hours
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/peak_purchase_hours';

CREATE TABLE IF NOT EXISTS golden_monthly_trend
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/monthly_trend';

CREATE TABLE IF NOT EXISTS golden_weekend_vs_weekday
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/weekend_vs_weekday';

CREATE TABLE IF NOT EXISTS golden_revenue_per_session
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/revenue_per_session';

CREATE TABLE IF NOT EXISTS golden_cart_abandonment_sessions
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/cart_abandonment_sessions';

CREATE TABLE IF NOT EXISTS golden_user_purchase_frequency
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/user_purchase_frequency';

CREATE TABLE IF NOT EXISTS golden_category_revenue
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/category_revenue';

CREATE TABLE IF NOT EXISTS golden_view_to_purchase_time
USING DELTA
LOCATION '/mnt/govinda/goldenlayers/view_to_purchase_time';



In [0]:
%sql
show tables

database tableName isTemporary default abandoned_products false default brand_conversion_funnel false default brand_daily_trend false default brand_retention false default cart_abandonment_sessions false default category_revenue false default dayofweek_activity false default global_conversion_funnel false default golden_abandoned_products false default golden_brand_conversion_funnel false default golden_brand_daily_trend false default golden_brand_retention false default golden_cart_abandonment_sessions false default golden_category_revenue false default golden_dayofweek_activity false default golden_global_conversion_funnel false default golden_high_revenue_products false default golden_hourly_activity false default golden_monthly_trend false default golden_peak_purchase_hours false default golden_price_band_distribution false default golden_product_lifespan false default golden_repeat_customers false default golden_revenue_per_session false default golden_session_durations false default golden_sessions_no_purchase false default golden_top_brands_by_revenue false default golden_top_brands_by_volume false default golden_top_products_by_sales false default golden_top_spenders false default golden_user_purchase_frequency false default golden_view_to_purchase_time false default golden_weekend_vs_weekday false default high_revenue_products false default hourly_activity false default monthly_trend false default t1 false default t2 false default v1 false _sqldf true